# ParameterServerStrategy on Hops
---

<font color='red'> <h3>Tested with TensorFlow 1.11</h3></font>

In [ ]:
def distributed_training():

    import sys
    import numpy as np
    import tensorflow as tf
    import os
    
    from hops import tensorboard
    from hops import devices

    def input_fn():
      x = np.random.random((1024, 10))
      y = np.random.randint(2, size=(1024, 1))
      x = tf.cast(x, tf.float32)
      dataset = tf.data.Dataset.from_tensor_slices((x, y))
      dataset = dataset.repeat(500)
      dataset = dataset.batch(32)
      return dataset

    model_dir = tensorboard.logdir()
    print('Using %s to store checkpoints.' % model_dir)

    # Define a Keras Model.
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(10,)))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model.
    optimizer = tf.train.GradientDescentOptimizer(0.2)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    model.summary()
    tf.keras.backend.set_learning_phase(True)

    # Define DistributionStrategies and convert the Keras Model to an
    # Estimator that utilizes these DistributionStrateges.
    # Evaluator is a single worker, so using MirroredStrategy.
    run_config = tf.estimator.RunConfig(
        experimental_distribute=tf.contrib.distribute.DistributeConfig(
            train_distribute=tf.contrib.distribute.ParameterServerStrategy(
                num_gpus_per_worker=devices.get_num_gpus()),
            eval_distribute=tf.contrib.distribute.MirroredStrategy(
                num_gpus_per_worker=devices.get_num_gpus())))
    keras_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, config=run_config, model_dir=model_dir)

    # Train and evaluate the model. Evaluation will be skipped if there is not an
    # "evaluator" job in the cluster.
    tf.estimator.train_and_evaluate(keras_estimator, train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
        eval_spec=tf.estimator.EvalSpec(input_fn=input_fn))
    
from hops import experiment
experiment.parameter_server(distributed_training)